In [9]:
!pip install requests



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Complete Code**

In [1]:
import requests
import csv

# Your GitHub API Token
GITHUB_API_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

# Set headers for API authentication
headers = {
    'Authorization': f'token {GITHUB_API_TOKEN}'
}

# Function to get users in Mumbai with more than 50 followers
def get_users_in_mumbai(min_followers=50):
    url = f'https://api.github.com/search/users?q=location:mumbai+followers:>{min_followers}'
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        users = response.json()['items']
        return users
    else:
        print(f"Failed to fetch users: {response.status_code}")
        return []

# Function to get detailed information about a specific user
def get_user_details(username):
    url = f'https://api.github.com/users/{username}'
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch details for user {username}: {response.status_code}")
        return {}

# Function to get repositories for a specific user
def get_user_repositories(username):
    url = f'https://api.github.com/users/{username}/repos'
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch repos for user {username}: {response.status_code}")
        return []

# Function to write users to a CSV file
def write_users_to_csv(users_data):
    with open('users.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=[
            'login', 'name', 'company', 'location', 'email', 
            'hireable', 'bio', 'public_repos', 'followers', 
            'following', 'created_at'
        ])
        writer.writeheader()
        for user in users_data:
            writer.writerow(user)

# Function to write repositories to a CSV file
def write_repositories_to_csv(repos_data):
    with open('repositories.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=[
            'login', 'full_name', 'created_at', 'stargazers_count', 
            'watchers_count', 'language', 'has_projects', 'has_wiki', 
            'license_name'
        ])
        writer.writeheader()
        for repo in repos_data:
            writer.writerow(repo)

# Main function
def main():
    # Step 1: Fetch users in Mumbai with more than 50 followers
    users = get_users_in_mumbai(min_followers=50)
    users_data = []
    repos_data = []

    # Step 2: Loop through each user to get detailed user info and repositories
    for user in users:
        user_details = get_user_details(user['login'])
        if user_details:
            # Clean and structure user data
            cleaned_user = {
                'login': user_details.get('login', ''),
                'name': user_details.get('name', ''),
                'company': user_details.get('company', '') if user_details.get('company') else '',  # Handle None value
                'location': user_details.get('location', ''),
                'email': user_details.get('email', ''),
                'hireable': user_details.get('hireable', ''),
                'bio': user_details.get('bio', ''),
                'public_repos': user_details.get('public_repos', ''),
                'followers': user_details.get('followers', ''),
                'following': user_details.get('following', ''),
                'created_at': user_details.get('created_at', '')
            }
            # Append the cleaned user data to the list
            users_data.append(cleaned_user)

            # Step 3: Fetch the repositories for each user
            repos = get_user_repositories(user['login'])
            for repo in repos:
                # Clean and structure repository data
                repos_data.append({
                    'login': user['login'],
                    'full_name': repo.get('full_name', ''),
                    'created_at': repo.get('created_at', ''),
                    'stargazers_count': repo.get('stargazers_count', 0),
                    'watchers_count': repo.get('watchers_count', 0),
                    'language': repo.get('language', ''),
                    'has_projects': repo.get('has_projects', False),
                    'has_wiki': repo.get('has_wiki', False),
                    # Handle the NoneType license issue by checking if 'license' exists
                    'license_name': repo['license']['key'] if repo.get('license') else ''
                })

    # Step 4: Write user data to 'users.csv'
    write_users_to_csv(users_data)
    
    # Step 5: Write repositories data to 'repositories.csv'
    write_repositories_to_csv(repos_data)

# Run the main function
if __name__ == "__main__":
    main()


In [2]:
users = get_users_in_mumbai(min_followers=50)
print(f"Number of users fetched: {len(users)}")


Number of users fetched: 30
